# LNG Cargo Diversion Decision Engine - Step by Step

## What We'll Build

You're an LNG trader with a cargo headed to **Europe (TTF)**, but **Asia (JKM)** prices are higher.

**Should you divert the cargo?**

We'll calculate:
1. **Netback** for each destination (profit after costs)
2. **Decision** (DIVERT or KEEP)
3. **Trade Ticket** (hedge sizing)
4. **Risk Analysis** (stress testing)

## Formula Reference

**Full documentation**: See `docs/formulas.md` for all formulas and sources

### Physical Constants
- LNG Density: 0.45 tonnes/m³
- Energy Content: 52 MMBtu/tonne LNG
- CO₂ Factor: 3.114 tCO₂/tonne fuel
- Boil-off Rate: 0.10% per day (industry standard for modern TFDE vessels)

### Key Formulas
- **Voyage Time**: `Days = Distance(nm) / (Speed(knots) × 24)`
- **Boil-off**: `Loss = Cargo × Rate × Days`
- **Netback**: `Revenue - Fuel Cost - Charter Cost - Carbon Cost`
- **Decision**: `Adjusted ΔNetback ≥ Threshold`

### Sources
- Vessel specs: Industry standard for 174k m³ TFDE carriers
- Market data: Platts JKM/TTF, Baltic Exchange freight
- Carbon: EU ETS (EUA) pricing

In [2]:
pip install yfinance

  Using cached yfinance-1.1.0-py2.py3-none-any.whl.metadata (6.1 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached peewee-3.19.0-py3-none-any.whl.metadata (7.0 kB)
Using cached yfinance-1.1.0-py2.py3-none-any.whl (129 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.1 MB/s  0:00:00
Using cached peewee-3.19.0-py3-none-any.whl (411 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [yfinance]7/8 [yfinance]]]

[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timezone
import numpy as np
import matplotlib.pyplot as plt

In [9]:
def _last_close(ticker: str, lookback_days: int = 14) -> float:
   """
   Pull last available close from Yahoo Finance.
   No API key required. Raises ValueError if no data returned.
   """
   df = yf.download(ticker, period=f"{lookback_days}d", interval="1d", progress=False)
   if df is None or df.empty:
       raise ValueError(f"No data returned for ticker {ticker}")
   
   # Handle both single-level and multi-level columns
   if isinstance(df.columns, pd.MultiIndex):
       close = df["Close"].iloc[:, 0].dropna()
   else:
       close = df["Close"].dropna()
   
   if close.empty:
       raise ValueError(f"No Close prices found for ticker {ticker}")
   return float(close.iloc[-1])

In [ ]:
 # True real time JKM and freight data require expensive subscriptions (Platts, Baltic Exchange), this function uses a workaround: pull free market data where available (TTF, EUA) and estimate the rest using reasonable proxies based on typical market relationships. This allows the notebook to run with live ish data without API keys or subscriptions.
def get_market_snapshot(
   *,
   ttf_ticker: str = "TTF=F",
   eua_ticker: str = "CO2.L",
   asia_premium_usd_per_mmbtu: float = 2.50,     # proxy 
   freight_regime: str = "normal",               # "loose" oversupplied shipping market, rates $20,000/day lower than baseline |"normal" balanced market, baseline rate $85,000/day by default |"tight" high demand, low supply, rates $35,000/day higher than baseline
   base_freight_usd_day: float = 85000,          # proxy 
   fuel_price_usd_per_t: float = 747             # proxy LNG bunker (boil-off gas fuel). Modern TFDE LNG carriers burn LNG boil-off as fuel rather than VLSFO/MGO. This is the market rate for LNG bunker fuel (~$747/MT vs VLSFO ~$583/MT). Using LNG bunker price because: 1) TFDE vessels use boil-off for propulsion 2) More accurate for operational costs 3) Reflects actual LNG carrier economics
) -> dict:
   """
   Hybrid snapshot:
   - Real-ish: TTF, EUA from Yahoo Finance (no key)
   - Proxy: JKM, freight, fuel
   """
   ttf = _last_close(ttf_ticker)
   eua = _last_close(eua_ticker)
# JKM proxy
   jkm = ttf + asia_premium_usd_per_mmbtu
   
# Freight proxy (very simple regimes)
   regime_bump = {"loose": -20000, "normal": 0, "tight": 35000}
   if freight_regime not in regime_bump:
       raise ValueError("freight_regime must be one of: loose, normal, tight")
   freight = base_freight_usd_day + regime_bump[freight_regime]
   return {
       "asof_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
       "ttf_price_usd_per_mmbtu": ttf,
       "eua_price_proxy_usd_per_tco2": eua,
       "jkm_price_proxy_usd_per_mmbtu": jkm,
       "freight_rate_proxy_usd_day": freight,
       "fuel_price_proxy_usd_per_t": fuel_price_usd_per_t,
       "meta": {
           "ttf_source": f"Yahoo Finance {ttf_ticker}",
           "eua_source": f"Yahoo Finance {eua_ticker}",
           "jkm_method": "TTF + asia_premium (proxy)",
           "freight_method": "regime-based proxy",
           "fuel_method": "LNG bunker proxy (TFDE boil-off fuel)"
       }
   }

mkt = get_market_snapshot(
   asia_premium_usd_per_mmbtu=2.75,
   freight_regime="normal",
)

# Variables for engine input
ttf_price = mkt["ttf_price_usd_per_mmbtu"]
jkm_price = mkt["jkm_price_proxy_usd_per_mmbtu"]
freight_rate = mkt["freight_rate_proxy_usd_day"]
fuel_price = mkt["fuel_price_proxy_usd_per_t"]
eua_price = mkt["eua_price_proxy_usd_per_tco2"]
print("MARKET SNAPSHOT:", mkt["asof_utc"])
print(mkt["meta"])
print(f"TTF={ttf_price:.2f}, JKM(proxy)={jkm_price:.2f}, Freight(proxy)={freight_rate:,.0f}/day, Fuel(LNG bunker)={fuel_price:.0f}/MT, EUA={eua_price:.2f}")

MARKET SNAPSHOT: 2026-02-08T23:26:38+00:00
{'ttf_source': 'Yahoo Finance TTF=F', 'eua_source': 'Yahoo Finance CO2.L', 'jkm_method': 'TTF + asia_premium (proxy)', 'freight_method': 'regime-based proxy', 'fuel_method': 'fixed proxy input'}
TTF=35.69, JKM(proxy)=38.44, Freight(proxy)=85,000/day, EUA=74.40


## Define Parameters

Let's set up our ship and route parameters:
- **Ship capacity**: 174,000 m³ of LNG
- **Speed**: 19.5 knots
- **Boil-off**: 0.10% per day (LNG evaporates!)
- **Routes**: US Gulf → Rotterdam (5,000 nm) vs US Gulf → Tokyo (9,500 nm)

In [3]:
# Vessel parameters
cargo_capacity_m3 = 174000       # Ship holds 174,000 cubic meters
laden_speed_kn = 19.5            # Ship speed when loaded
boil_off_pct_per_day = 0.10      # 0.10% evaporates daily
fuel_consumption_tpd = 130       # Burns 130 tonnes fuel/day

# Route distances (nautical miles)
distance_europe_nm = 5000        # US Gulf → Rotterdam
distance_asia_nm = 9500          # US Gulf → Tokyo

# Conversion constants
LNG_DENSITY = 0.45               # 1 m³ LNG = 0.45 tonnes
ENERGY_PER_TONNE = 52            # 1 tonne LNG = 52 MMBtu
CO2_FACTOR = 3.114               # 1 tonne fuel = 3.114 tonnes CO2

print(" Parameters loaded!")
print(f"   Europe distance: {distance_europe_nm:,} nm")
print(f"   Asia distance: {distance_asia_nm:,} nm")

 Parameters loaded!
   Europe distance: 5,000 nm
   Asia distance: 9,500 nm


## Calculate Voyage Time 

How long does each voyage take?

**Formula**: `Voyage Days = Distance / (Speed x 24 hours)`

In [4]:
# Calculate voyage duration
hours_per_day = 24
europe_voyage_days = distance_europe_nm / (laden_speed_kn * hours_per_day)
asia_voyage_days = distance_asia_nm / (laden_speed_kn * hours_per_day)

print(" VOYAGE TIME")
print(f"   To Europe: {europe_voyage_days:.1f} days")
print(f"   To Asia:   {asia_voyage_days:.1f} days")
print(f"   Extra time to Asia: {asia_voyage_days - europe_voyage_days:.1f} days")
print(f"\n    Asia is {((asia_voyage_days-europe_voyage_days)/europe_voyage_days)*100:.0f}% longer")

 VOYAGE TIME
   To Europe: 10.7 days
   To Asia:   20.3 days
   Extra time to Asia: 9.6 days

    Asia is 90% longer


## Calculate Boil-Off 

LNG is stored at -162°C. Even with insulation, some evaporates every day.

**Formula**: 
- `Boil-Off = Cargo × Boil-Off Rate × Voyage Days`
- `Delivered Cargo = Original Cargo − Boil-Off`

In [5]:
# Calculate boil-off for each destination
europe_boiloff_m3 = cargo_capacity_m3 * (boil_off_pct_per_day / 100) * europe_voyage_days
asia_boiloff_m3 = cargo_capacity_m3 * (boil_off_pct_per_day / 100) * asia_voyage_days

# Delivered cargo
europe_delivered_m3 = cargo_capacity_m3 - europe_boiloff_m3
asia_delivered_m3 = cargo_capacity_m3 - asia_boiloff_m3

# Convert to energy (MMBtu)
europe_delivered_mmbtu = europe_delivered_m3 * LNG_DENSITY * ENERGY_PER_TONNE
asia_delivered_mmbtu = asia_delivered_m3 * LNG_DENSITY * ENERGY_PER_TONNE

print(" BOIL-OFF LOSS")
print(f"   Europe: {europe_boiloff_m3:,.0f} m³ lost ({europe_boiloff_m3/cargo_capacity_m3*100:.2f}%)")
print(f"   Asia:   {asia_boiloff_m3:,.0f} m³ lost ({asia_boiloff_m3/cargo_capacity_m3*100:.2f}%)")

print(f"\n DELIVERED ENERGY")
print(f"   To Europe: {europe_delivered_mmbtu:,.0f} MMBtu")
print(f"   To Asia:   {asia_delivered_mmbtu:,.0f} MMBtu")
print(f"   Lost to Asia: {europe_delivered_mmbtu - asia_delivered_mmbtu:,.0f} MMBtu")

 BOIL-OFF LOSS
   Europe: 1,859 m³ lost (1.07%)
   Asia:   3,532 m³ lost (2.03%)

 DELIVERED ENERGY
   To Europe: 4,028,100 MMBtu
   To Asia:   3,988,950 MMBtu
   Lost to Asia: 39,150 MMBtu


## Calculate Voyage Costs 

Three main costs:
1. **Fuel Cost** = Fuel consumed × Fuel price
2. **Charter Cost** = Days × Daily freight rate
3. **Carbon Cost** = CO₂ emissions × EUA price

In [ ]:
# Today's market prices
ttf_price = 35.69         # Europe gas price ($/MMBtu)
jkm_price = 38.44         # Asia gas price ($/MMBtu)
freight_rate = 85000      # Charter cost ($/day)
fuel_price = 583          # Proxy (VLSFO or LNG bunker proxy)
eua_price = 74.40         # Carbon price ($/tCO2)


# EUROPE COSTS
europe_fuel_tonnes = fuel_consumption_tpd * europe_voyage_days
europe_fuel_cost = europe_fuel_tonnes * fuel_price
europe_charter_cost = freight_rate * europe_voyage_days
europe_carbon_cost = europe_fuel_tonnes * CO2_FACTOR * eua_price
europe_total_cost = europe_fuel_cost + europe_charter_cost + europe_carbon_cost

# ASIA COSTS
asia_fuel_tonnes = fuel_consumption_tpd * asia_voyage_days
asia_fuel_cost = asia_fuel_tonnes * fuel_price
asia_charter_cost = freight_rate * asia_voyage_days
asia_carbon_cost = asia_fuel_tonnes * CO2_FACTOR * eua_price
asia_total_cost = asia_fuel_cost + asia_charter_cost + asia_carbon_cost

print(" VOYAGE COSTS")
print(f"   Europe: ${europe_total_cost:,.0f}")
print(f"   Asia:   ${asia_total_cost:,.0f}")
print(f"   Extra cost to Asia: ${asia_total_cost - europe_total_cost:,.0f}")

NameError: name 'fuel_consumption_tpd' is not defined

## Calculate Netback 

**Netback** = Revenue from selling LNG − All voyage costs

This is the profit you make after paying for fuel, charter, and carbon.

**Formula**: `Netback = (Delivered Energy × Gas Price) − Voyage Costs`

In [7]:
# REVENUE: Sell the delivered LNG at market prices
europe_revenue = europe_delivered_mmbtu * ttf_price
asia_revenue = asia_delivered_mmbtu * jkm_price

# NETBACK: Revenue minus all costs
europe_netback = europe_revenue - europe_total_cost
asia_netback = asia_revenue - asia_total_cost

# UPLIFT: How much more money do you make going to Asia?
netback_uplift = asia_netback - europe_netback

print(" REVENUE")
print(f"   Europe: ${europe_revenue:,.0f}")
print(f"   Asia:   ${asia_revenue:,.0f}")

print(f"\n NETBACK (Profit)")
print(f"   Europe: ${europe_netback:,.0f}")
print(f"   Asia:   ${asia_netback:,.0f}")

print(f"\n UPLIFT")
print(f"   Asia makes ${netback_uplift:,.0f} MORE than Europe")
print(f"   That's a {netback_uplift/europe_netback*100:.1f}% profit increase!")

 REVENUE
   Europe: $50,955,465
   Asia:   $61,030,935

 NETBACK (Profit)
   Europe: $48,853,972
   Asia:   $57,038,099

 UPLIFT
   Asia makes $8,184,127 MORE than Europe
   That's a 16.8% profit increase!


## Apply Decision Rules 

Real trading isn't just "which makes more money?" You need safety buffers:

1. **Basis Adjustment** (5%): Account for price basis risk and execution slippage
2. **Decision Threshold** ($500k): Minimum uplift to justify operational complexity

**Formula**:
Adjusted Uplift = Uplift × (1 − Basis %) Decision = DIVERT if Adjusted Uplift ≥ Threshold, else KEEP

In [8]:
# Decision rules
basis_adjustment_pct = 5.0      # 5% haircut for basis risk
decision_threshold = 500000     # Need at least $500k uplift

# Apply basis adjustment
adjusted_uplift = netback_uplift * (1 - basis_adjustment_pct / 100)

# Make decision
should_divert = adjusted_uplift >= decision_threshold
decision = "DIVERT" if should_divert else "KEEP"

print(" DECISION LOGIC")
print(f"   Raw uplift: ${netback_uplift:,.0f}")
print(f"   Basis adjustment ({basis_adjustment_pct}%): -${netback_uplift * basis_adjustment_pct/100:,.0f}")
print(f"   Adjusted uplift: ${adjusted_uplift:,.0f}")
print(f"   Threshold: ${decision_threshold:,.0f}")
print(f"\n{'='*50}")
print(f"   DECISION: {decision} Yes" if should_divert else f"   DECISION: {decision} No")
print(f"{'='*50}")

 DECISION LOGIC
   Raw uplift: $8,184,127
   Basis adjustment (5.0%): -$409,206
   Adjusted uplift: $7,774,920
   Threshold: $500,000

   DECISION: DIVERT Yes


## Generate Trade Ticket 

When you divert, you must hedge the price exposure:

1. **Sell TTF futures** = Cancel your original Europe hedge
2. **Buy JKM futures** = Lock in the Asia price

**Hedge Sizing**:
- Contract size: 10,000 MMBtu per lot
- Lots = Delivered Energy / 10,000 MMBtu (rounded)

In [11]:
# Hedge sizing
contract_size_mmbtu = 10000

# Calculate number of futures contracts needed
ttf_lots = round(europe_delivered_mmbtu / contract_size_mmbtu)
jkm_lots = round(asia_delivered_mmbtu / contract_size_mmbtu)

print("TRADE TICKET")
print("="*50)
print(f"   Sell {ttf_lots} lots TTF @ ${ttf_price:.2f}/MMBtu")
print(f"   Buy  {jkm_lots} lots JKM @ ${jkm_price:.2f}/MMBtu")
print("="*50)
print(f"\n   Total exposure: ${abs(jkm_lots * jkm_price - ttf_lots * ttf_price) * contract_size_mmbtu:,.0f}")
print(f"   Expected P&L: ${adjusted_uplift:,.0f}")

TRADE TICKET
   Sell 403 lots TTF @ $12.65/MMBtu
   Buy  399 lots JKM @ $15.30/MMBtu

   Total exposure: $10,067,500
   Expected P&L: $7,774,920


## Stress Test the Decision 

What if market prices change? Let's test scenarios:

1. **Price Shocks**: JKM/TTF move ±10%
2. **Freight Spike**: Charter rates double
3. **Combined Worst Case**: All risks hit at once

**Goal**: Ensure the decision is robust, not marginal.

In [15]:
# Stress test scenarios
price_shock_pct = 10.0
freight_multiplier = 2.0

print(" STRESS TESTING")
print("="*50)

# Scenario 1: JKM drops 10%
jkm_stressed = jkm_price * (1 - price_shock_pct / 100)
asia_revenue_stressed = asia_delivered_mmbtu * jkm_stressed
asia_netback_stressed = asia_revenue_stressed - asia_total_cost
uplift_stressed = (asia_netback_stressed - europe_netback) * 0.95
print(f"\n1 JKM drops {price_shock_pct}% to ${jkm_stressed:.2f}")
print(f"   Uplift: ${uplift_stressed:,.0f}")
print(f"   Decision: {'DIVERT ' if uplift_stressed >= decision_threshold else 'KEEP '}")

# Scenario 2: TTF rises 10%
ttf_stressed = ttf_price * (1 + price_shock_pct / 100)
europe_revenue_stressed = europe_delivered_mmbtu * ttf_stressed
europe_netback_stressed = europe_revenue_stressed - europe_total_cost
uplift_stressed2 = (asia_netback - europe_netback_stressed) * 0.95
print(f"\n2 TTF rises {price_shock_pct}% to ${ttf_stressed:.2f}")
print(f"   Uplift: ${uplift_stressed2:,.0f}")
print(f"   Decision: {'DIVERT ' if uplift_stressed2 >= decision_threshold else 'KEEP'}")

# Scenario 3: Freight doubles
freight_stressed = freight_rate * freight_multiplier
asia_charter_stressed = freight_stressed * asia_voyage_days
asia_total_cost_stressed = asia_fuel_cost + asia_charter_stressed + asia_carbon_cost
asia_netback_stressed3 = asia_revenue - asia_total_cost_stressed
uplift_stressed3 = (asia_netback_stressed3 - europe_netback) * 0.95
print(f"\n3 Freight doubles to ${freight_stressed:,.0f}/day")
print(f"   Uplift: ${uplift_stressed3:,.0f}")
print(f"   Decision: {'DIVERT ' if uplift_stressed3 >= decision_threshold else 'KEEP '}")

# Scenario 4: Combined worst case
jkm_worst = jkm_price * 0.90
ttf_worst = ttf_price * 1.10
freight_worst = freight_rate * 2.0
europe_rev_worst = europe_delivered_mmbtu * ttf_worst
asia_rev_worst = asia_delivered_mmbtu * jkm_worst
asia_cost_worst = asia_fuel_cost + (freight_worst * asia_voyage_days) + asia_carbon_cost
uplift_worst = ((asia_rev_worst - asia_cost_worst) - (europe_rev_worst - europe_total_cost)) * 0.95
print(f"\n4  WORST CASE: All risks combined")
print(f"   Uplift: ${uplift_worst:,.0f}")
print(f"   Decision: {'DIVERT ' if uplift_worst >= decision_threshold else 'KEEP'}")

print("\n" + "="*60)

 STRESS TESTING

1 JKM drops 10.0% to $13.77
   Uplift: $1,976,982
   Decision: DIVERT 

2 TTF rises 10.0% to $13.92
   Uplift: $2,934,151
   Decision: DIVERT 

3 Freight doubles to $173,000/day
   Uplift: $6,106,838
   Decision: DIVERT 

4  WORST CASE: All risks combined
   Uplift: $-4,531,870
   Decision: KEEP



##  Summary 

You've built a complete LNG cargo diversion decision engine!

### Learnings:
1.  Physical calculations (boil-off, energy conversion)
2.  Voyage economics (fuel, charter, carbon costs)
3.  Netback analysis (revenue minus all costs)
4.  Risk management (basis adjustment, thresholds)
5.  Trade execution (hedge sizing with futures)
6.  Stress testing (scenario analysis)

### Next Steps:
- Run with different market prices (change `ttf_price`, `jkm_price`, etc.)
- Try different vessels (smaller/larger capacity)
- Add more routes (Middle East, West Africa)
- Compare this notebook to the full engine in `engine/` folder
- Run the CLI: `python app.py --date 2026-01-20`
